In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
# Paths to your datasets
correct_squat_1_path = "Clean_Dataset/Correct_Squat_1.xlsx"
correct_squat_2_path = "Clean_Dataset/Correct_Squat_2.xlsx"
incorrect_squat_1_path = "Clean_Dataset/Incorrect_Squat_1.xlsx"
incorrect_squat_2_path = "Clean_Dataset/Incorrect_Squat_2.xlsx"

# Load the datasets
correct_squat_1 = pd.read_excel(correct_squat_1_path)
correct_squat_2 = pd.read_excel(correct_squat_2_path)
incorrect_squat_1 = pd.read_excel(incorrect_squat_1_path)
incorrect_squat_2 = pd.read_excel(incorrect_squat_2_path)

# Optionally add a 'Label' column if your datasets don't already include it
correct_squat_1['Label'] = 1
correct_squat_2['Label'] = 1
incorrect_squat_1['Label'] = 0
incorrect_squat_2['Label'] = 0

# Combine the datasets
all_data = pd.concat([correct_squat_1, correct_squat_2, incorrect_squat_1, incorrect_squat_2], ignore_index=True)

features = all_data.drop('Label', axis=1)
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

##### Data Processing 

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

features = all_data.drop('Label', axis=1)
labels = all_data['Label']
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features_normalized, labels, test_size=0.2, random_state=42)

##### Define and Train the Neural Network Models

In [4]:
from keras.models import Sequential
from keras.layers import Dense

def create_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = create_model(X_train.shape[1])
model.fit(X_train, y_train, epochs=10, validation_split=0.1)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/data_adapter.py:1808: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


Epoch 1/10
31/31 [==============================] - 1s 9ms/step - loss: 0.6593 - accuracy: 0.5595 - val_loss: 0.6344 - val_accuracy: 0.5872
Epoch 2/10
31/31 [==============================] - 0s 3ms/step - loss: 0.6131 - accuracy: 0.6057 - val_loss: 0.5980 - val_accuracy: 0.6514
Epoch 3/10
31/31 [==============================] - 0s 3ms/step - loss: 0.5770 - accuracy: 0.6869 - val_loss: 0.5572 - val_accuracy: 0.7523
Epoch 4/10
31/31 [==============================] - 0s 3ms/step - loss: 0.5345 - accuracy: 0.7710 - val_loss: 0.5086 - val_accuracy: 0.8257
Epoch 5/10
31/31 [==============================] - 0s 3ms/step - loss: 0.4911 - accuracy: 0.8193 - val_loss: 0.4617 - val_accuracy: 0.8624
Epoch 6/10
31/31 [==============================] - 0s 2ms/step - loss: 0.4470 - accuracy: 0.8409 - val_loss: 0.4177 - val_accuracy: 0.8532
Epoch 7/10
31/31 [==============================] - 0s 3ms/step - loss: 0.4131 - accuracy: 0.8593 - val_loss: 0.3811 - val_accuracy: 0.8716
Epoch 8/10
31/31 [==

##### Printing performance

In [5]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

9/9 [==============================] - 0s 2ms/step - loss: 0.3096 - accuracy: 0.8967
Test Accuracy: 0.8966789841651917


In [6]:
from keras.models import load_model
model.save('Sequential_model.h5')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
